## traditional

In [1]:
import numpy as np

def make_cluster(natoms, radius=20, seed=1981):
    np.random.seed(seed)
    arr = np.random.normal(0, radius, size=(natoms,3))-0.5
    return arr

In [2]:
class lj_pure(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot


    @classmethod
    def distance(cls, atom1, atom2):
        dx = atom2[0] - atom1[0]
        dy = atom2[1] - atom1[1]
        dz = atom2[2] - atom1[2]

        r = (dx*dx + dy*dy + dz*dz)**0.5
        return r


    @classmethod
    def potential(cls, cluster):
        energy = 0.0
        for i in range(len(cluster)-1):
            for j in range(i+1,len(cluster)):
                r = cls.distance(cluster[i],cluster[j])
                e = cls.lj(r)
                energy += e
        return energy

In [3]:
import numpy as np
class lj_numpy(object):
    
    @classmethod
    def lj(cls, r):
        sr6 = (1./r)**6
        pot = 4.*(sr6*sr6 - sr6)
        return pot
    
    
    @classmethod
    def distances(cls, cluster):
        diff = cluster[:, np.newaxis, :] - cluster[np.newaxis, :, :]
        mat = np.sqrt((diff*diff).sum(-1))
        return mat

    
    @classmethod
    def potential(cls, cluster):
        d = cls.distances(cluster)
        pot = cls.lj(d)
        energy = np.nansum(pot) / 2
        return energy

In [4]:
cluster = make_cluster(int(2e3), radius=100)

In [5]:
%timeit lj_pure.potential(cluster)

4 s ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit lj_numpy.potential(cluster)

/Users/adefusco/Applications/miniconda3/envs/intel/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: divide by zero encountered in true_divide
/Users/adefusco/Applications/miniconda3/envs/intel/lib/python3.6/site-packages/ipykernel/__main__.py:7: RuntimeWarning: invalid value encountered in subtract


305 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## numba

In [28]:
import numba

@numba.vectorize(['float64(float64)'], nopython=True, target='parallel')
def ulj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot

@numba.jit(nopython=True)
def dist(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.guvectorize(['(float64[:,:], float64[:,:])'], '(n,m)->(n,n)', nopython=True, target='parallel')
def distance_matrix(cluster, dmat):
    for i in range(len(cluster)-1):
        dmat[i,i] = 0.0
        for j in range(i+1,len(cluster)):
            r = dist(cluster[i],cluster[j])
            dmat[i,j] = r
            dmat[j,i] = r

def upotential(cluster):
    n = cluster.shape[0]
    dmat = np.empty(shape=(n,n), dtype=cluster.dtype)
    distance_matrix(cluster, dmat)
    
    pot = ulj(dmat)
    energy = np.nansum(pot) / 2.
    
    return energy

In [30]:
%timeit upotential(cluster)

/Users/adefusco/Applications/miniconda3/envs/intel/lib/python3.6/site-packages/ipykernel/__main__.py:33: RuntimeWarning: divide by zero encountered in ulj
/Users/adefusco/Applications/miniconda3/envs/intel/lib/python3.6/site-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in ulj


34.6 ms ± 569 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## thanks, Intel!

https://www.anaconda.com/blog/developer-blog/parallel-python-with-numba-and-parallelaccelerator/

In [18]:
import numba

@numba.jit(nopython=True)
def lj(r):
    sr6 = (1./r)**6
    pot = 4.*(sr6*sr6 - sr6)
    return pot


@numba.jit(nopython=True)
def distance(atom1, atom2):
    dx = atom2[0] - atom1[0]
    dy = atom2[1] - atom1[1]
    dz = atom2[2] - atom1[2]

    r = np.sqrt(dx*dx + dy*dy + dz*dz)
    return r


@numba.jit(nopython=True)
def potential(cluster):
    energy = 0.0
    for i in range(len(cluster)-1):
        for j in range(i+1,len(cluster)):
            r = distance(cluster[i],cluster[j])
            e = lj(r)
            energy += e
    return energy

In [19]:
z = np.zeros((1,3))
%time potential(z)

CPU times: user 141 ms, sys: 3.58 ms, total: 144 ms
Wall time: 143 ms


0.0

In [20]:
%timeit potential(cluster)

14.6 ms ± 91.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## bigger data structures

In [12]:
import xarray as xr

In [13]:
a = list('HCNO')
labels = np.random.choice(list('HCNO'), size=cluster.shape[0])

traj = np.stack([cluster, cluster+0.001, cluster-0.02])

In [14]:
positions = xr.DataArray(traj,
                        dims=('time','atom','position'),
                        coords={'atom':labels,
                                'position':['x','y','z'],
                                'time':range(traj.shape[0])})
positions

<xarray.DataArray (time: 3, atom: 2000, position: 3)>
array([[[ -32.180493,  -49.396232,  172.05197 ],
        [-100.648437,  105.830152,  212.067684],
        ..., 
        [ 167.829985,   93.934129,   13.338662],
        [  36.553448,  124.789454, -156.322434]],

       [[ -32.179493,  -49.395232,  172.05297 ],
        [-100.647437,  105.831152,  212.068684],
        ..., 
        [ 167.830985,   93.935129,   13.339662],
        [  36.554448,  124.790454, -156.321434]],

       [[ -32.200493,  -49.416232,  172.03197 ],
        [-100.668437,  105.810152,  212.047684],
        ..., 
        [ 167.809985,   93.914129,   13.318662],
        [  36.533448,  124.769454, -156.342434]]])
Coordinates:
  * atom      (atom) <U1 'C' 'O' 'C' 'O' 'O' 'N' 'O' 'C' 'H' 'H' 'O' 'O' 'C' ...
  * position  (position) <U1 'x' 'y' 'z'
  * time      (time) int64 0 1 2

In [15]:
positions.sel(atom='C')

<xarray.DataArray (time: 3, atom: 471, position: 3)>
array([[[ -32.180493,  -49.396232,  172.05197 ],
        [ -96.074573,   31.574305,  -63.347429],
        ..., 
        [ -24.140257,  233.775356,   28.248137],
        [-143.094791,  136.85085 , -131.000658]],

       [[ -32.179493,  -49.395232,  172.05297 ],
        [ -96.073573,   31.575305,  -63.346429],
        ..., 
        [ -24.139257,  233.776356,   28.249137],
        [-143.093791,  136.85185 , -130.999658]],

       [[ -32.200493,  -49.416232,  172.03197 ],
        [ -96.094573,   31.554305,  -63.367429],
        ..., 
        [ -24.160257,  233.755356,   28.228137],
        [-143.114791,  136.83085 , -131.020658]]])
Coordinates:
  * atom      (atom) <U1 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' ...
  * position  (position) <U1 'x' 'y' 'z'
  * time      (time) int64 0 1 2

In [37]:
%timeit potential(positions.sel(time=0).values)

15.2 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
